# 25 Pandas Functions You Didn't Know Existed | P(Guarantee) = 0.8
## ExcelWriter, factorize, explode, squeeze, T, mask, idxmax, clip, ...
![](https://images.pexels.com/photos/5199661/pexels-photo-5199661.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@introspectivedsgn?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Erik Mclean</a>
        on 
        <a href='https://www.pexels.com/photo/unrecognizable-man-in-panda-head-sitting-near-car-5199661/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a> All images are by the author unless specified otherwise.
    </strong>
</figcaption>

## Setup

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.display.max_rows = None
warnings.filterwarnings("ignore")

## Introduction

"I wish I could do this operation in Pandas…."

Well, chances are, you can!

Pandas is so vast and deep that it enables you to execute virtually any tabular manipulation you can think of. However, this vastness sometimes comes at a disadvantage.

Many elegant features that solve rare edge-cases, unique scenarios are lost in the documentation, shadowed by the more frequently used functions.

This kernel aims to rediscover those features and show you that Pandas is more capable than you ever knew.

## 1. `ExcelWriter`

`ExcelWriter` is a generic class for creating excel files (with sheets!) and writing DataFrames to them. Let's say we have these 2:

In [ ]:
# Load two datasets
diamonds = sns.load_dataset("diamonds")
tips = sns.load_dataset("tips")

```python
# Write to the same excel file
with pd.ExcelWriter("data.xlsx") as writer:

    diamonds.to_excel(writer, sheet_name="diamonds")
    tips.to_excel(writer, sheet_name="tips")
```

It has additional attributes to specify the DateTime format to be used, whether you want to create a new excel file or modify an existing one, what happens when a sheet exists, etc. Check out the details from the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html).

## 2. `pipe`

![](https://images.pexels.com/photos/192473/pexels-photo-192473.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@leah-kelley-50725?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Leah Kelley</a>
        on 
        <a href='https://www.pexels.com/photo/grayscale-photo-of-man-holding-tobacco-pipe-192473/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

`pipe` is one of the best functions for doing data cleaning in a concise, compact manner in Pandas. It allows you to chain multiple custom functions into a single operation.

For example, let's say you have functions to `drop_duplicates`, `remove_outliers`, `encode_categoricals` that accept their own arguments. Here is how you apply all three in a single operation:

```python
df_preprocessed = (diamonds.pipe(drop_duplicates).
                            pipe(remove_outliers, ['price', 'carat', 'depth']).
                            pipe(encode_categoricals, ['cut', 'color', 'clarity'])
                  )
```

I like how this function resembles [Sklearn pipelines](https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d). There is more you can do with it, so check out the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html) or this [helpful article](https://towardsdatascience.com/a-better-way-for-data-preprocessing-pandas-pipe-a08336a012bc).

## 3. `factorize`

This function is a pandas alternative to Sklearn's `LabelEncoder`:

In [ ]:
# Mind the [0] at the end
diamonds["cut_enc"] = pd.factorize(diamonds["cut"])[0]

diamonds["cut_enc"].sample(5)

Unlike `LabelEncoder`, `factorize` returns a tuple of two values: the encoded column and a list of the unique categories:

In [ ]:
codes, unique = pd.factorize(diamonds["cut"], sort=True)

codes[:10]

In [ ]:
unique

## 4. `explode` - 🤯🤯

![](https://images.unsplash.com/photo-1567446042109-8a62d37fea07?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=750&q=80)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@joshuas?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Joshua Sukoff</a>
        on 
        <a href='https://unsplash.com/s/photos/explode?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>

A function with an interesting name is `explode`. Let's see an example first and then, explain:

In [ ]:
data = pd.Series([1, 6, 7, [46, 56, 49], 45, [15, 10, 12]]).to_frame("dirty")
data

The `dirty` column has two rows where values are recorded as actual lists. You may often see this type of data in surveys as some questions accept multiple answers.

In [ ]:
data.explode("dirty", ignore_index=True)

`explode` takes a cell with an array-like value and explodes it into multiple rows. Set `ignore_index` to True to keep the ordering of a numeric index.

## 5. `squeeze`

![](https://images.pexels.com/photos/5875701/pexels-photo-5875701.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@cottonbro?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>cottonbro</a>
        on 
        <a href='https://www.pexels.com/photo/close-up-photo-of-sausage-5875701/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a>
    </strong>
</figcaption>

Another function with a funky name is `squeeze` and is used in very rare but annoying edge cases.

One of these cases is when a single value is returned from a condition used to subset a DataFrame. Consider this example:

In [ ]:
subset = diamonds.loc[diamonds.index < 1, ["price"]]
subset

Even though there is just one cell, it is returned as a DataFrame. This can be annoying since you now have to use `.loc` again with both the column name and index to access the price.

But, if you know `squeeze`, you don't have to. The function enables you to remove an axis from a single-cell DataFrame or Series. For example:

In [ ]:
subset.squeeze()

Now, only the scalar is returned. It is also possible to specify the axis to remove:

In [ ]:
subset.squeeze("columns")  # or "rows"

Note that `squeeze` only works for DataFrames or Series with single values.

## 6. between

![](https://images.unsplash.com/photo-1596806082495-fb05004cfe49?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=769&q=80)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@jujudreaminx?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Justin Dream</a>
        on 
        <a href='https://unsplash.com/s/photos/between?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Pexels</a>
    </strong>
</figcaption>

A rather nifty function for boolean indexing numeric features within a range:

In [ ]:
# Get diamonds that are priced between 3500 and 3700 dollars
diamonds[diamonds["price"].between(3500, 3700, inclusive="neither")].sample(5)

## 7. `T`

![](https://images.pexels.com/photos/258083/pexels-photo-258083.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@pixabay?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pixabay</a>
        on 
        <a href='https://www.pexels.com/photo/close-up-of-illuminated-text-against-black-background-258083/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

All DataFrames have a simple `T` attribute, which stands for transpose. You may not use it often, but I find it quite useful when displaying DataFrames of the `describe` method:

In [ ]:
## HIDE
from sklearn.datasets import load_boston

bunch = load_boston()
boston = pd.DataFrame(bunch["data"], columns=bunch["feature_names"])

In [ ]:
boston.describe().T.head(10)

The Boston housing dataset has 30 numeric columns. If you call `describe` as-is, the DataFrame will stretch horizontally, making it hard to compare the statistics. Taking the transpose will switch the axes so that summary statistics are given in columns.

## 8. Pandas Styler

Did you know that Pandas allows you to style DataFrames?

They have a `style` attribute which opens doors to customizations and styles only limited by your HTML and CSS knowledge. I won't discuss the full details of what you can do with `style` but only show you my favorite functions:

In [ ]:
## HIDE
diabetes = pd.read_csv("https://raw.githubusercontent.com/BexTuychiev/medium_stories/master/2021/august/1_pandas_funcs/data/diabetes.csv")

In [ ]:
diabetes.describe().T.drop("count", axis=1).style.highlight_max(color="darkred")

Above, we are highlighting cells that hold the maximum value of a column. Another cool styler is `background_gradient` which can give columns a gradient background color based on their values:

In [ ]:
diabetes.describe().T.drop("count", axis=1).style.background_gradient(
    subset=["mean", "50%"], cmap="Reds"
)

This feature comes especially handy when you are using `describe` on a table with many columns and want to compare summary statistics. Check out the documentation of the styler [here](https://pandas.pydata.org/docs/reference/style.html).

## 9. Pandas options

Like Matplotlib, pandas has global settings that you can tweak to change the default behaviors:

In [ ]:
dir(pd.options)

These settings are divided into 5 modules. Let's see what settings are there under `display`:

In [ ]:
dir(pd.options.display)

There are options under `display` but I mostly use `max_columns` and `precision`:

In [ ]:
# Remove the limit to display the number of cols
pd.options.display.max_columns = None

# Only show 5 numbers after the decimal
pd.options.display.precision = 5  # gets rid of scientific notation

You can check out the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html) to dig deeper into this wonderful feature.

## 10. `convert_dtypes`

We all know that pandas has an annoying tendency to mark some columns as `object` data type. Instead of manually specifying their types, you can use `convert_dtypes` method which tries to infer the best data type:

In [ ]:
sample = pd.read_csv(
    "https://raw.githubusercontent.com/BexTuychiev/medium_stories/master/2021/august/1_pandas_funcs/data/station_day.csv",
    usecols=["StationId", "CO", "O3", "AQI_Bucket"],
)
sample.dtypes

In [ ]:
sample.convert_dtypes().dtypes

Unfortunately, it can't pares dates due to the caveats of different date time formats.

## 11. `select_dtypes`

A function I use all the time is `select_dtypes`. I think it is obvious what the function does from its name. It has `include` and `exclude` parameters that you can use to select columns including or excluding certain data types.

For example, choose only numeric columns with `np.number`:

In [ ]:
# Choose only numerical columns
diamonds.select_dtypes(include=np.number).head()

Or `exclude` them:

In [ ]:
# Exclude numerical columns
diamonds.select_dtypes(exclude=np.number).head()

## 12. `mask`

![](https://images.pexels.com/photos/38275/anonymous-studio-figure-photography-facial-mask-38275.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@pixabay?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pixabay</a>
        on 
        <a href='https://www.pexels.com/photo/photo-of-guy-fawkes-mask-with-red-flower-on-top-on-hand-38275/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a>
    </strong>
</figcaption>

`mask` allows you to quickly replace cell values where a custom condition is true. 

For example, let's say we have a survey data collected from people aged 50-60.

In [ ]:
# Create sample data
ages = pd.Series([55, 52, 50, 66, 57, 59, 49, 60]).to_frame("ages")

ages

We will treat ages that are outside 50-60 range (there are two, 49 and 66) as data entry mistakes and replace them with NaNs.

In [ ]:
ages.mask(cond=~ages["ages"].between(50, 60), other=np.nan)

So, `mask` replaces values that don't meet `cond` with `other`.

## 13. `min` and `max` along the columns axis

Even though `min` and `max` functions are well-known, they have another useful property for some edge-cases. Consider this dataset:

In [ ]:
index = ["Diamonds", "Titanic", "Iris", "Heart Disease", "Loan Default"]
libraries = ["XGBoost", "CatBoost", "LightGBM", "Sklearn GB"]
df = pd.DataFrame(
    {lib: np.random.uniform(90, 100, 5) for lib in libraries}, index=index
)

df

The above fake DataFrame is a point-performance of 4 different gradient boosting libraries on 5 datasets. We want to find the library that performed best at each dataset. Here is how you do it elegantly with `max`:

In [ ]:
df.max(axis=1)

Just change the axis to 1 and you get a row-wise max/min. 

## 14. `nlargest` and `nsmallest`

Sometimes you don't just want the min/max of a column. You want to see the top N or ~(top N) values of a variable. This is where `nlargest` and `nsmallest` comes in handy.

Let's see the top 5 most expensive and cheapest diamonds:

In [ ]:
diamonds.nlargest(5, "price")

In [ ]:
diamonds.nsmallest(5, "price")

## 15. `idxmax` and `idxmin`

When you call `max` or `min` on a column, pandas returns the value that is largest/smallest. However, sometimes you want the *position* of the min/max, which is not possible with these functions.

Instead, you should use `idxmax`/`idxmin`:

In [ ]:
diamonds.price.idxmax()

In [ ]:
diamonds.carat.idxmin()

You can also specify the `columns` axis, in which case the functions return the index number of the column.

## 16. `value_counts` with `dropna=False`

A common operation to find the percentage of missing values in a column is to chain `isnull` and `sum` and divide by the length of the array. 

But, you can do the same thing with `value_counts` with relevant arguments:

In [ ]:
ames_housing = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

ames_housing["FireplaceQu"].value_counts(dropna=False, normalize=True)

Fireplace quality of Ames housing dataset consists of 47% nulls.

## 17. `clip`

![](https://images.pexels.com/photos/2448452/pexels-photo-2448452.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@ann-h-45017?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Ann H</a>
        on 
        <a href='https://www.pexels.com/photo/a-lot-of-paper-clips-2448452/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a></strong>
</figcaption>

Outlier detection and removal is common in data analysis. 

`clip` function makes it really easy to find outliers outside a range and replacing them with the hard limits. 

Let's go back to the ages example:

In [ ]:
ages

This time, we will replace the out-of-range ages with the hard limits of 50 and 60:

In [ ]:
ages.clip(50, 60)

Fast and efficient!

## 18. `at_time` and `between_time`

These two can be useful when working with time-series that have high granularity. 

`at_time` allows you to subset values at a specific date or time. Consider this time series:

In [ ]:
index = pd.date_range("2021-08-01", periods=100, freq="H")
data = pd.DataFrame({"col": list(range(100))}, index=index)

data.head()

Let's select all rows at 3 PM:

In [ ]:
data.at_time("15:00")

Cool, huh? Now, let's use `between_time` to select rows within a custom interval:

In [ ]:
from datetime import datetime

data.between_time("09:45", "12:00")

Note that both functions require a DateTimeIndex and they only work with times (as in *o'clock*). If you want to subset within a DateTime interval, use `between`.

## 19. `bdate_range`

`bdate_range` is a short-hand function to create TimeSeries indices with business-day frequency:

In [ ]:
series = pd.bdate_range("2021-01-01", "2021-01-31")  # A period of one month
len(series)

Business-day frequencies are common in the financial world. So, this function may come in handy when reindexing existing time-series with `reindex` function.

## 20. `autocorr`

One of the critical components in time-series analysis is examining the autocorrelation of a variable. 

Autocorrelation is the plain-old correlation coefficient but it is calculated with the lagging version of a time series. 

In more detail, autocorrelation of a time series at `lag=k` is calculated as follows:

1. The time-series is shifted till `k` periods:

In [ ]:
## HIDE
# Prep the data for an example
dt = pd.date_range("2021-01-01", periods=len(tips))
tips.index = dt

time_series = tips[["tip"]]

In [ ]:
time_series["lag_1"] = time_series["tip"].shift(1)
time_series["lag_2"] = time_series["tip"].shift(2)
time_series["lag_3"] = time_series["tip"].shift(3)
time_series["lag_4"] = time_series["tip"].shift(4)
# time_series['lag_k'] = time_series['tip'].shift(k)

time_series.head()

2. Correlation is calculated between the original `tip` and each `lag_*`. 

Instead of doing all this manually, you can use the `autocorr` function of Pandas:

In [ ]:
# Autocorrelation of tip at lag_10
time_series["tip"].autocorr(lag=8)

You can read more about the importance of autocorrelation in time-series analysis from this [post](https://towardsdatascience.com/advanced-time-series-analysis-in-python-decomposition-autocorrelation-115aa64f475e).

## 21. `hasnans`

Pandas offers a quick method to check if a given series contains any nulls with `hasnans` attribute:

In [ ]:
series = pd.Series([2, 4, 6, "sadf", np.nan])
series.hasnans

According to its [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.hasnans.html), it enables various performance increases. Note that the attribute works only on `pd.Series`.

## 22. `at` and `iat`

These two accessors are much faster alternatives to `loc` and `iloc` with a disadvantage. They only allow selecting or replacing a single value at a time:

In [ ]:
# [index, label]
diamonds.at[234, "cut"]

In [ ]:
# [index, index]
diamonds.iat[1564, 4]

In [ ]:
# Replace 16541th row of the price column
diamonds.at[16541, "price"] = 10000

## 23. `argsort`

You should use this function when you want to extract the indices that would sort an array:

In [ ]:
tips.reset_index(inplace=True, drop=True)

sort_idx = tips["total_bill"].argsort(kind="mergesort")

# Now, sort `tips` based on total_bill
tips.iloc[sort_idx].head()

## 24. `cat` accessor

It is common knowledge that Pandas enables to use built-in Python functions on dates and strings using accessors like `dt` or `str`. 

Pandas also has a special `category` data type for categorical variables as can be seen below:

In [ ]:
diamonds.dtypes

When a column is `category`, you can use several special functions using the `cat` accessor. For example, let's see the unique categories of diamond cuts:

In [ ]:
diamonds["cut"].cat.categories

There are also functions like `remove_categories` or `rename_categories`, etc.:

In [ ]:
diamonds["new_cuts"] = diamonds["cut"].cat.rename_categories(list("ABCDE"))
diamonds["new_cuts"].cat.categories

You can see the full list of functions under the `cat` accessor [here](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#categorical-accessor).

## 25. `GroupBy.nth`

This function only works with `GroupBy` objects. Specifically, after grouping, `nth` returns the nth row from each group:

In [ ]:
diamonds.groupby("cut").nth(5)

## Summary

Even though libraries like Dask and datatable are slowly winning over Pandas with their shiny new features for handling massive datasets, Pandas still remains the most widely-used data manipulation tool in Python data science ecosystem.

The library still remains as a role-model for other packages to imitate and improve upon, as it integrates into the modern SciPy stack so well. Thank you for reading!

## You might also be interested...
- [My 6-part Powerful EDA Template](https://www.kaggle.com/bextuychiev/my-6-part-powerful-eda-template)
- [
Lasso regression with Pipelines (Tutorial)](https://www.kaggle.com/bextuychiev/lasso-regression-with-pipelines-tutorial)
- [Awesome EDA + XGBoost CV Baseline](https://www.kaggle.com/bextuychiev/relevant-eda-xgboost-cv-baseline)